### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_distances

from lightfm.datasets import fetch_movielens

/usr/local/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1526    Hercules (1997)',
 '360    Lion King, The (1994)',
 '2692    Iron Giant, The (1999)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [13]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [15]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1884    French Connection, The (1971)',
 '1892    Rain Man (1988)',
 '453    Fugitive, The (1993)',
 '957    African Queen, The (1951)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не используя готовые решения, реализовать SVD разложение, используя SGD на explicit данных

In [63]:
users = ratings.user_id
movies = ratings.movie_id
rat = ratings.rating
user_item_exp = sp.coo_matrix((rat, (users, movies)))
user_item_csr = user_item_exp.tocsr()

In [116]:
class MF:
    def __init__(self):
        self.W = None
        self.H = None
    
    def similar_items(self, item_id, n=10):
        dist = cosine_distances(self.H.T, self.H.T)
        return [(id_, d) for id_, d in zip(np.argsort(dist[item_id]), np.sort(dist[item_id]))][:n]
    
    def similar_users(self, user_id, n=10):
        dist = cosine_distances(self.W, self.W)
        return [(id_, d) for id_, d in zip(np.argsort(dist[user_id]), np.sort(dist[user_id]))][:n]
    
    def recommend(self, user_id, user_item_csr, n=10):
        pred = np.argsort(self.W[user_id] @ self.H)
        rated = set(user_item_csr[user_id].indices)
        
        cnt = 0
        output = []
        for el in pred:
            if cnt == n:
                break
            if el not in rated:
                output.append([el])
            cnt += 1
        return output
    
    def mse(self, V):
        pred = np.array([self.W[i, :] @ self.H[:, j] for i, j in zip(*V.nonzero())])
        return np.mean(np.square(V.data - pred))

In [84]:
class SVD(MF):
    def __init__(self, features, users, items):
        super().__init__()
        self.k = features
        self.users = users
        self.items = items
        self.W = np.random.uniform(0, 1 / np.sqrt(self.k), (users, self.k))
        self.H = np.random.uniform(0, 1 / np.sqrt(self.k), (self.k, items))
        self.bu = np.random.uniform(0, 1 / np.sqrt(self.k), (users, 1))
        self.bi = np.random.uniform(0, 1 / np.sqrt(self.k), (items, 1))
        
    def fit(self, ratings, batch_size=500, iterations=1000, lr=1e-3, lambda_=1e-3):
        V = ratings
#         self.mu = V[np.nonzero(V)].mean()
        
        user_nonzero, item_nonzero = V.nonzero()
        
        for iteration in tqdm(range(iterations)):
            # чтобы итерация была проходом по нескольким значениям 
            # (могут быть повторения за счет случайности)
            for value in tqdm(np.random.permutation(user_nonzero.shape[0]), position=0, leave=True):
#                 i, j = np.random.randint(0, self.users), np.random.randint(0, self.items)
                i, j = user_nonzero[value], item_nonzero[value]
#                 error = (self.W[i, :] @ self.H[:, j] + self.mu + self.bu[i] + self.bi[j]) - V[i, j]
                error = self.W[i, :] @ self.H[:, j] - V[i, j]
                self.W[i, :] -= lr * (error * self.H[:, j].T + lambda_ * self.W[i, :])
                self.H[:, j] -= lr * (error * self.W[i, :].T + lambda_ * self.H[:, j])
                
#                 self.bu[i] -= lr * (error + lambda_ * self.bu[i])
#                 self.bi[j] -= lr * (error + lambda_ * self.bi[j])
            
            print(f'Iteration {iteration} MSE = {self.mse(V)}')
            # convergence
            if iteration % 1000 == 0 and np.allclose(V.toarray(), self.predict()):
                return
    
    def predict(self):
        return self.W @ self.H #+ self.mu + self.bu + self.bi.T

In [86]:
%%time
users, items = user_item_csr.shape
svd = SVD(64, users, items)

CPU times: user 14.1 ms, sys: 2.2 ms, total: 16.3 ms
Wall time: 14.7 ms


In [87]:
%%time
svd.fit(user_item_csr, iterations=10, lr=1e-2, lambda_=0.1)

CPU times: user 1.59 ms, sys: 625 µs, total: 2.22 ms
Wall time: 1.18 ms


In [88]:
get_similars(1, svd)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '2009    Jungle Book, The (1967)',
 "2286    Bug's Life, A (1998)",
 '1628    Witness (1985)',
 '2327    Shakespeare in Love (1998)',
 '1287    When Harry Met Sally... (1989)',
 '3294    American Graffiti (1973)',
 '3457    Parenthood (1989)',
 "941    It's a Wonderful Life (1946)"]

### Задание 2. Не используя готовые решения, реализовать матричное разложение, используя ALS на implicit данных

In [140]:
class ALS(MF):
    def __init__(self, features, users, items):
        super().__init__()
        self.k = features 
        self.W = np.random.uniform(0, 1 / np.sqrt(self.k), (users, self.k))
        self.H = np.random.uniform(0, 1 / np.sqrt(self.k), (self.k, items))
    
    def fit(self, ratings, lr=1e-1, iterations=10, log_every=10):
        R = ratings
        for iteration in range(iterations):
            self.H = np.linalg.inv(self.W.T @ self.W + lr * np.eye(self.k)) @ self.W.T @ R
            self.W = (np.linalg.inv(self.H @ self.H.T + lr * np.eye(self.k)) @ self.H @ R.T).T
            
            if iteration % log_every == 0 or iteration + 1 == iterations:
                print(f'Iteration {iteration + 1} MSE = {self.mse(ratings)}')

In [138]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [141]:
users, items = user_item_csr.shape
als = ALS(64, users, items)
als.fit(user_item_csr, iterations=100)

Iteration 1 MSE = 0.5291843469888944
Iteration 11 MSE = 0.4085729151206799
Iteration 21 MSE = 0.40815655627602043
Iteration 31 MSE = 0.4080260014679035
Iteration 41 MSE = 0.40796271858751876
Iteration 51 MSE = 0.4079183605495556
Iteration 61 MSE = 0.40788271943409704
Iteration 71 MSE = 0.40785270378636923
Iteration 81 MSE = 0.4078268344052093
Iteration 91 MSE = 0.4078041601954663
Iteration 100 MSE = 0.4077859030411155


In [142]:
get_similars(1, als)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '2252    Pleasantville (1998)',
 '360    Lion King, The (1994)',
 '2692    Iron Giant, The (1999)',
 '1526    Hercules (1997)']

### Задание 3. Не используя готовые решения, реализовать матричное разложение BPR на implicit данных

In [184]:
class BPR(MF):
    def __init__(self, features, users, items):
        super().__init__()
        self.k = features 
        self.users = users
        self.items = items
        self.W = np.random.uniform(0, 1 / np.sqrt(self.k), (users, self.k))
        self.H = np.random.uniform(0, 1 / np.sqrt(self.k), (self.k, items))
    
    def sigmoid(self, x):
        return np.exp(-x) / (1 + np.exp(-x))
    
    def fit(self, ratings, iterations, lr=1e-2, lambda_=0.1, eps=1e-8):
        V = ratings
        
        user_nonzero, item_nonzero = V.nonzero()
        print(f'Start MSE = {self.mse(ratings)}')
        for iteration in tqdm(range(iterations), position=0, leave=True):
            for value in tqdm(np.random.permutation(user_nonzero.shape[0]), position=0, leave=True):
                i, j = user_nonzero[value], item_nonzero[value]
                # taking one random negative sample
                negative_sample = np.random.choice(np.setdiff1d(np.arange(self.items), V[i].nonzero()[1]))
                x_uij = self.W[i, :] @ (self.H[:, j] - self.H[:, negative_sample])
                sigma = self.sigmoid(x_uij + eps)
                
                self.W[i,:] += lr * \
                        (sigma * (self.H[:, j] - self.H[:, negative_sample]) + lambda_ * self.W[i, :])
                self.H[:, j] += lr * \
                        (sigma * self.W[i, :] + lambda_ * self.H[:, j])
                self.H[:, negative_sample] += lr * \
                        (-sigma * self.W[i, :] + lambda_ * self.H[:, negative_sample])
                
            print(f'Iteration {iteration} MSE = {self.mse(ratings)}')

In [185]:
users, items = user_item_csr.shape
bpr = BPR(64, users, items)
bpr.fit(user_item_csr, iterations=6, lr=1e-3, lambda_=1e-3)

  0%|          | 191/575281 [00:00<05:01, 1906.89it/s]

Start MSE = 0.5603821827704101


  0%|          | 206/575281 [00:00<04:39, 2054.43it/s]

Iteration 0 MSE = 0.46682400406634234


  0%|          | 188/575281 [00:00<05:06, 1875.86it/s]

...

Iteration 3 MSE = 0.269561916761583


  0%|          | 183/575281 [00:00<05:14, 1827.83it/s]

Iteration 4 MSE = 0.24262369531302946


100%|██████████| 6/6 [31:16<00:00, 312.80s/it]

Iteration 5 MSE = 0.24443393161999408


In [190]:
get_similars(1, bpr)

['0    Toy Story (1995)',
 '293    Pulp Fiction (1994)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2789    American Beauty (1999)',
 '2502    Matrix, The (1999)',
 '1959    Saving Private Ryan (1998)',
 '537    Blade Runner (1982)',
 '847    Godfather, The (1972)',
 '589    Silence of the Lambs, The (1991)',
 '3106    Galaxy Quest (1999)']

### Задание 4. Не используя готовые решения, реализовать матричное разложение WARP на implicit данных

In [186]:
class WARP(MF):
    def __init__(self, features, users, items, m=1):
        super().__init__()
        self.k = features 
        self.users = users
        self.items = items
        self.m = m
        self.W = np.random.uniform(0, 1 / np.sqrt(self.k), (users, self.k))
        self.H = np.random.uniform(0, 1 / np.sqrt(self.k), (self.k, items))
    
    def fit(self, ratings, iterations, lr, lambda_, negatives=10):
        V = ratings
        
        user_nonzero, item_nonzero = V.nonzero()
        print(f'Start MSE = {self.mse(ratings)}')
        for iteration in tqdm(range(iterations), position=0, leave=True):
            for value in tqdm(np.random.permutation(user_nonzero.shape[0]), position=0, leave=True):
                i, j = user_nonzero[value], item_nonzero[value]
                positive = self.W[i, :] @ self.H[:, j]
                for neg in range(1, negatives + 1):
                    negative_sample = np.random.choice(np.setdiff1d(np.arange(self.items), V[i].nonzero()[1]))
                    negative = self.W[negative_sample, :] @ self.H[:, negative_sample]
                    if self.m  + negative - positive > 0:
                        loss = np.log(negatives // neg)
                        
                        self.W[i, :] -= lr * (loss * (self.H[:, negative_sample] - self.H[:, j]) 
                                              + lambda_ * self.W[i, :])
                        self.H[:, j] -= lr * (-loss * self.W[i, :] + lambda_ * self.H[:, j])
                        self.H[:, negative_sample] -= lr * (loss * self.W[i, :] 
                                                            + lambda_ * self.H[:, negative_sample])
            
            print(f'Iteration {iteration} MSE = {self.mse(ratings)}')

In [ ]:
users, items = user_item_csr.shape
warp = WARP(64, users, items)
warp.fit(user_item_csr, iterations=3, lr=1e-4, lambda_=1e-3)

In [189]:
get_similars(1, warp)

['0    Toy Story (1995)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '476    Jurassic Park (1993)',
 '2789    American Beauty (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '604    Fargo (1996)',
 '2693    Sixth Sense, The (1999)',
 '1180    Raiders of the Lost Ark (1981)',
 '1220    Terminator, The (1984)',
 "1176    One Flew Over the Cuckoo's Nest (1975)"]